In [1]:
import torch
from torch.autograd import Variable
from torch import nn, optim
from torch.nn import init

---

## 1. 标准RNN

In [2]:
# 建立简单的循环神经网络

basic_rnn = nn.RNN(input_size=3, hidden_size=5, num_layers=2)

In [3]:
# 访问第一层网络的 $w_{ih}$

basic_rnn.weight_ih_l0

Parameter containing:
 0.0763 -0.0092 -0.1030
-0.1201 -0.2317  0.3021
-0.1568 -0.0355  0.3281
-0.3040 -0.1278  0.2782
-0.2930 -0.1496  0.2063
[torch.FloatTensor of size 5x3]

In [4]:
# 访问第二层的 $w_{ih}$

basic_rnn.weight_ih_l1

Parameter containing:
 0.2065  0.3279 -0.0407 -0.1670  0.2792
-0.2776 -0.3277  0.2683  0.0603  0.1309
-0.3711  0.3196  0.4190  0.0326 -0.3182
-0.0235 -0.0512  0.3191 -0.1126  0.0228
 0.2597 -0.2444  0.3148 -0.0795 -0.1418
[torch.FloatTensor of size 5x5]

In [5]:
# 访问第一层的 $w_{hh}$

basic_rnn.weight_hh_l1

Parameter containing:
 0.2150  0.0983  0.0374  0.2345 -0.4368
 0.4327 -0.0015  0.4188  0.0039 -0.2717
 0.4181 -0.2944 -0.0375 -0.3815 -0.3615
 0.1330 -0.4197 -0.3870 -0.2852  0.3714
 0.0946 -0.0085 -0.4026 -0.1688  0.2727
[torch.FloatTensor of size 5x5]

In [6]:
# 访问第一层的 $b_{ih}$

basic_rnn.bias_ih_l0

Parameter containing:
-0.0978
 0.2153
 0.2239
 0.2170
 0.0324
[torch.FloatTensor of size 5]

In [7]:
# 输入的序列长度为5，批量是10，维度是3
toy_input = Variable(torch.randn(5, 10, 3))

# 初始隐藏状态 layer * direction = 2, 批量是10，维度是5
h_0 = Variable(torch.randn(2, 10, 5))

In [8]:
toy_output, h_n = basic_rnn(toy_input, h_0)

# toy_output的形状应该为 (5, 10, 5)
# h_n 的形状应该为 (2, 10, 5)
print(toy_output.shape)
print(h_n.shape)

torch.Size([5, 10, 5])
torch.Size([2, 10, 5])


---

## 2. LSTM

In [9]:
lstm = nn.LSTM(input_size=3, hidden_size=5, num_layers=2)

LSTM中间比标准RNN多了三个线性变换，多的三个线性变换的权重拼在一起，所以一共是4倍，同理偏置也是4倍。
换句话说，LSTM里面做了4个类似标准RNN所做的运算，所以参数个数是标准RNN的4倍。

In [10]:
# 参数的大小将变成 (4 * hidden_size, input_size) = (4 × 5, 3) = (20, 3)

lstm.weight_ih_l0

Parameter containing:
-0.0370  0.3933  0.4261
 0.0104  0.2447  0.2633
-0.3971  0.1586  0.2650
 0.1254  0.0325 -0.0926
-0.1548 -0.2009 -0.1171
-0.1834  0.3234  0.0946
-0.3815  0.0475  0.3884
-0.2534 -0.2354  0.2124
-0.4156 -0.1013 -0.3804
-0.1959  0.2669  0.0208
-0.1414 -0.0864 -0.0238
 0.4439 -0.2000 -0.0276
-0.3399  0.2919  0.0363
-0.0171  0.1917  0.4374
 0.3896  0.0809 -0.4040
-0.3879 -0.3903  0.1277
 0.1634  0.3729  0.1317
-0.2193  0.1497  0.0905
 0.1066 -0.2967  0.0568
-0.2763  0.0103  0.3772
[torch.FloatTensor of size 20x3]

In [11]:
# toy_input的size为 (5, 10, 3)

lstm_out, (h_n, c_n) = lstm(toy_input)

In [12]:
# lstm的输出矩阵应该是 (5, 10, 5)
print('lstm_out_size = ', lstm_out.size())

# h_n 和 c_n 的size应为 (2, 10, 5)
print('h_n_size = ', h_n.size())
print('c_n_size = ', c_n.size())

lstm_out_size =  torch.Size([5, 10, 5])
h_n_size =  torch.Size([2, 10, 5])
c_n_size =  torch.Size([2, 10, 5])


---

## 3. GRU
1. GRU的隐藏状态数量为标准RNN的3倍；
2. 网络的隐藏状态不是 $h_0 \text{和} c_0$，而是只有 $h_0$；
3. 其余部分和LSTM相同；

---
## 4. 单步版本
Pytorch提供 `RNNCell`, `LSTMCell`, `GRUCell`分别作为这三个模型的单步版本。

它们的输入不再是一个序列，而是一个序列中的一步，也就是循环神经网络的一个循环。

单步版本在序列的应用上更加灵活，能在基础上添加更多的自定义操作。